In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

In [ ]:
calls = pd.read_csv('/content/drive/MyDrive/United data set/callsf0d4f5a.csv')
reason = pd.read_csv('/content/drive/MyDrive/United data set/reason18315ff.csv')
sentiment = pd.read_csv('/content/drive/MyDrive/United data set/sentiment_statisticscc1e57a.csv')
customers = pd.read_csv('/content/drive/MyDrive/United data set/customers2afd6ea.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# existing reasons
reasons1 = reason['primary_call_reason'].unique()
reason['primary_call_reason'] = reason['primary_call_reason'].str.strip()
reason['primary_call_reason'] = reason['primary_call_reason'].replace('\s+', ' ', regex=True)
reasons2 = reason['primary_call_reason'].unique()
print('first',reasons1, len(reasons1))
print('second',reasons2, len(reasons2))

# DATA PREPROCESSING: REASONS
As observed above we still have duplicates, like
'check in': 'check-in';'post flight': 'post-flight'; 'products and services': 'products & services' and 'voluntary cancel' must be 'voluntary cancellation' (for better presentation).

In [ ]:
# Converting all entries to lowercase to standardize case
reason['primary_call_reason'] = reason['primary_call_reason'].str.lower()

# Defining a dictionary for replacements (using lowercase keys)
replacements = {
    'check in': 'check-in',
    'post flight': 'post-flight',
    'products and services': 'products & services',
    'products & services': 'products & services',
    'voluntary cancel': 'voluntary cancellation',  # better presentation
    'mileage plus': 'mileageplus',  # Combine if necessary
}

# Applying the replacements
reason['primary_call_reason'] = reason['primary_call_reason'].replace(replacements)

# Now get the unique values again
reasons2 = reason['primary_call_reason'].unique()

print('Updated unique reasons:', reasons2, len(reasons2))

In [ ]:
# Unique reasons sorted alphabetically
sorted_reasons = sorted(reason['primary_call_reason'].unique())
print('Sorted unique reasons:', sorted_reasons, len(sorted_reasons))

# Data Preprocessing: Calls
The calls table is cleaned.
1. Missing values is checked.
2. Duplicates are dropped
3. Ensuring the data types
4. Conversion to datetime type
5. Invalid Call durations (call end before call start)
6. Filling missing transcripts with a placeholder

In [ ]:
# Checking missing values
print(calls.isnull().sum())

In [ ]:
# Checking missing values
calls.isnull().sum()

# Drop duplicates
calls.drop_duplicates(inplace=True)

# Ensuring the call_id, customer_id, agent_id are of correct data types
calls['call_id'] = calls['call_id'].astype(str)
calls['customer_id'] = calls['customer_id'].astype(str)
calls['agent_id'] = calls['agent_id'].astype(str)

# Converting datetime columns to datetime type
calls['call_start_datetime'] = pd.to_datetime(calls['call_start_datetime'])
calls['call_end_datetime'] = pd.to_datetime(calls['call_end_datetime'])
calls['agent_assigned_datetime'] = pd.to_datetime(calls['agent_assigned_datetime'])

# Checking for invalid or extreme duration values (e.g., end time before start time)
invalid_calls = calls[calls['call_end_datetime'] < calls['call_start_datetime']]
if not invalid_calls.empty:
    print("Invalid call durations:", invalid_calls)
    # Handle invalid records (e.g., drop or investigate further)
    calls = calls[calls['call_end_datetime'] >= calls['call_start_datetime']]


# Filling missing transcripts with a placeholder
calls['call_transcript'].fillna('No transcript available', inplace=True)

In [ ]:
print(calls.info())

In [ ]:
calls.head(10)

# **Data preprocessing of Reasons**

In [ ]:
# Check for missing values
print("Missing values before cleaning:", reason.isnull().sum())

# Drop duplicates if any
reason = reason.drop_duplicates()

# Ensure call_id is of the correct type
reason['call_id'] = reason['call_id'].astype(str)

# Clean primary_call_reason by normalizing text, trimming spaces, etc.
reason['primary_call_reason'] = reason['primary_call_reason'].str.lower().str.strip()
reason['primary_call_reason'] = reason['primary_call_reason'].replace({'&': 'and', '-': ' ', '\s+': ' '}, regex=True)

# Remove any special characters
reason['primary_call_reason'] = reason['primary_call_reason'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Handle missing values in primary_call_reason
reason['primary_call_reason'] = reason['primary_call_reason'].fillna('unknown reason')

# Check and print unique call reasons after cleaning
unique_reasons = reason['primary_call_reason'].unique()
print("Cleaned unique call reasons:", unique_reasons)

# inconsistencies in call reasons
reason['primary_call_reason'] = reason['primary_call_reason'].replace({
    'voluntary cancel': 'voluntary cancellation',
    'mileage plus': 'mileageplus',
    'traveler updates': 'traveler update',
    'check in': 'check-in',
    'post flight': 'post-flight'
})

unique_reasons_after_correction = reason['primary_call_reason'].unique()
print("Corrected unique call reasons:", unique_reasons_after_correction)

# **Data Preprocessing: Sentiment**
1. Missing Values
2. Removing Duplicates
3. Data types are correct
4. Clean 'agent_tone' and 'customer_tone' by normalizing text
5. Missing Sentiment Value and Silence percentage is replaced by its median value


In [ ]:
# Check for missing values
print("Missing values before cleaning:", sentiment.isnull().sum())

# Drop duplicates
sentiment = sentiment.drop_duplicates()

# Ensure 'call_id' and 'agent_id' are of the correct types
sentiment['call_id'] = sentiment['call_id'].astype(str)
sentiment['agent_id'] = sentiment['agent_id'].astype(str)

# Clean 'agent_tone' and 'customer_tone' by normalizing text (lowercase, trimming spaces)
sentiment['agent_tone'] = sentiment['agent_tone'].str.lower().str.strip()
sentiment['customer_tone'] = sentiment['customer_tone'].str.lower().str.strip()

# missing tones by filling with 'neutral'
sentiment['agent_tone'] = sentiment['agent_tone'].fillna('neutral')
sentiment['customer_tone'] = sentiment['customer_tone'].fillna('neutral')

# Check for invalid or extreme values in numeric columns
print("Sentiment numeric summary:")
print(sentiment[['average_sentiment', 'silence_percent_average']].describe())

# Cap extreme values in numeric columns
# 'average_sentiment' stays between 0 and 1, and 'silence_percent_average' between 0 and 100
sentiment['average_sentiment'] = sentiment['average_sentiment'].clip(lower=0, upper=1)
sentiment['silence_percent_average'] = sentiment['silence_percent_average'].clip(lower=0, upper=100)

# Handle missing sentiment values by replacing with median values
sentiment['average_sentiment'] = sentiment['average_sentiment'].fillna(sentiment['average_sentiment'].median())
sentiment['silence_percent_average'] = sentiment['silence_percent_average'].fillna(sentiment['silence_percent_average'].median())

# Re-checking for missing values after cleaning
print("Missing values after cleaning:", sentiment.isnull().sum())


In [ ]:
sentiment.head(10)

# **Tone Mapping: Customer and Agent**
Since there are only 5 values for tone and it has a hierarchy among them, it can be easily mapped to 5 integer values [-2,2].

In [ ]:
# Define a mapping for tone values
tone_mapping = {
    'polite': 2,
    'calm': 1,
    'neutral': 0,
    'frustrated': -1,
    'angry': -2
}

# Add the 'agent_tone_value' column by mapping the 'agent_tone' values to the corresponding numeric values
sentiment['agent_tone_value'] = sentiment['agent_tone'].map(tone_mapping)

# Add the 'customer_tone_value' column by mapping the 'customer_tone' values to the corresponding numeric values
sentiment['customer_tone_value'] = sentiment['customer_tone'].map(tone_mapping)

# If there are any tones not covered by the mapping, fill them with 0 (neutral)
sentiment['agent_tone_value'] = sentiment['agent_tone_value'].fillna(0)
sentiment['customer_tone_value'] = sentiment['customer_tone_value'].fillna(0)

# Check the updated sentiment DataFrame
print(sentiment[['agent_tone', 'agent_tone_value', 'customer_tone', 'customer_tone_value']].head())


## **Data Preprocessing: Customers**
1. Missing Values
2. Remove Duplicates
3. Customer_id: Data type -> String and Remove extra spaces
4. Missing data for elite_level_code: NaN = 0.0
5. Ensuring elite_level_code data type


In [ ]:
# Check for missing values
missing_values = customers.isnull().sum()
print("Missing values before cleaning:\n", missing_values)

# Drop duplicates
customers = customers.drop_duplicates()

# Ensure 'customer_id' is of the correct type (string)
customers['customer_id'] = customers['customer_id'].astype(str)

# Strip any extra spaces from 'customer_name' and ensure all names are in proper case
customers['customer_name'] = customers['customer_name'].str.strip().str.title()

# Handle missing 'elite_level_code' - Assuming NaN means non-elite (level 0), fill with 0.0
customers['elite_level_code'] = customers['elite_level_code'].fillna(0.0)

# Ensure 'elite_level_code' is of float type
customers['elite_level_code'] = customers['elite_level_code'].astype(float)

# Verify changes and check for missing values after cleaning
print("Missing values after cleaning:\n", customers.isnull().sum())
print(customers.head())

Merging all the data together for Analysis

In [ ]:
# merging all the columns to get a merged Data
merged_data = calls.merge(reason, on='call_id', how='left') \
                    .merge(sentiment, on='call_id', how='left') \
                    .merge(customers, on='customer_id', how='left')

# View the merged DataFrame
print("Merged DataFrame:")
print(merged_data.head())
print("\nColumns in Merged DataFrame:")
print(merged_data.columns)

### **AHT and AST calculation:**
Average Handling Time and Average Speed Time are the two parameters which will be calculated from the given Formula.
1. AHT = total_handle_time / total_calls
2. AST = total_waiting_time / total_calls

In [ ]:
# Calculate Call Duration in Minutes
# Call Duration = AST + AHT
merged_data['call_duration'] = (merged_data['call_end_datetime'] - merged_data['call_start_datetime']).dt.total_seconds() / 60  # duration in minutes

# Calculate AHT
# Calculate total handle time and total calls
total_handle_time = merged_data['call_duration'].sum()  # Total handle time in minutes
total_calls = merged_data['call_duration'].count()  # Total number of calls

# Calculate AHT
AHT = total_handle_time / total_calls


# Calculate AST (in minutes)
merged_data['waiting_time'] = (merged_data['agent_assigned_datetime'] - merged_data['call_start_datetime']).dt.total_seconds() / 60
total_waiting_time = merged_data['waiting_time'].sum()  # Sum of waiting times in minutes
total_calls = merged_data['waiting_time'].count()  # Total number of calls


AST = total_waiting_time / total_calls


print(f"Average Speed to Answer (AST): {AST:.2f} minutes")

print(f"Average Handle Time (AHT): {AHT:.2f} minutes")


In [ ]:
aht_data = merged_data.groupby('agent_id_x').agg(
    total_calls=('call_id', 'count'),  # Count total calls
    total_duration=('call_duration', 'sum')  # Total duration of all calls
).reset_index()

# Calculate AHT
aht_data['AHT'] = aht_data['total_duration'] / aht_data['total_calls']  # This gives AHT in seconds

merged_data = merged_data.merge(aht_data[['agent_id_x', 'AHT']], on='agent_id_x', how='left')

print(merged_data.head())

In [ ]:
merged_data.head()

In [ ]:
agent_id_col = 'agent_id_x'

# AHT and AST per agent

In [ ]:
# AHT and AST per agent
aht_ast_per_agent = merged_data.groupby(agent_id_col).agg({
    'waiting_time': 'mean',  # AST per agent
    'call_duration': 'mean',  # AHT per agent
    'call_id': 'count'  # Total calls handled per agent
}).reset_index().rename(columns={'waiting_time': 'AST', 'call_duration': 'AHT', 'call_id': 'total_calls'})

print(aht_ast_per_agent)

# AHT and AST per customer

In [ ]:
# AHT and AST per customer
aht_ast_per_customer = merged_data.groupby('customer_id').agg({
    'waiting_time': 'mean',  # AST per customer
    'call_duration': 'mean',  # AHT per customer
    'call_id': 'count'  # Total calls handled per customer
}).reset_index().rename(columns={'waiting_time': 'AST', 'call_duration': 'AHT', 'call_id': 'total_calls'})

print(aht_ast_per_customer)

# AHT and AST per call reason

In [ ]:
# AHT and AST per call reason
aht_ast_per_reason = merged_data.groupby('primary_call_reason').agg({
    'waiting_time': 'mean',  # AST per reason
    'call_duration': 'mean',  # AHT per reason
    'call_id': 'count'  # Total calls handled for each reason
}).reset_index().rename(columns={'waiting_time': 'AST', 'call_duration': 'AHT', 'call_id': 'total_calls'})

print(aht_ast_per_reason)

# Heat Map of AST, AHT and Total Calls by Call Reason

In [ ]:
# Create a heatmap for AHT and AST
plt.figure(figsize=(14, 8))
heatmap_data = aht_ast_per_reason.set_index('primary_call_reason')[['AST', 'AHT', 'total_calls']]

# Normalize values
normalized_data = (heatmap_data - heatmap_data.min()) / (heatmap_data.max() - heatmap_data.min())

sns.heatmap(normalized_data, annot=True, cmap='coolwarm', fmt='.2f', cbar_kws={'label': 'Normalized Values'})
plt.title('Heatmap of AST, AHT, and Total Calls by Call Reason', fontsize=16)
plt.xlabel('Metrics', fontsize=14)
plt.ylabel('Primary Call Reason', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# **Elite Customer Analysis**
For elite customers, i.e. frequent flyers are identified as those whose elite_level_code ranks in the top 10% of all customers. To determine this, the 90th percentile was calculated of the elite_level_code values, which serves as the benchmark for defining elite status. Any customer with an elite_level_code above this threshold is classified as elite. This approach helps isolate the most valuable customers, allowing the airline to tailor premium services and rewards to enhance their overall experience.

In [ ]:
customers.dropna(subset=['elite_level_code'], inplace=True)
# Calculate the 90th percentile for elite customers
percentile_90 = customers['elite_level_code'].quantile(0.9)
print(f"The 90th percentile benchmark elite_value_code is: {percentile_90}\n")

elite_customers = customers[customers['elite_level_code'] > percentile_90]

elite_customers = elite_customers[['customer_id', 'customer_name', 'elite_level_code']].drop_duplicates()

print("Elite Customers DataFrame:")
print(elite_customers)

Percentage of Elite Customers of Total Customers

In [ ]:
# Total number of customers
total_customers = len(customers)

# Number of elite customers
elite_customers_count = len(elite_customers)

# Calculate percentage of elite customers
elite_percentage = (elite_customers_count / total_customers) * 100

print(f"Percentage of Elite Customers: {elite_percentage:.2f}%")

In [ ]:
reason.head()

In [ ]:
elite_customers_data = (
    elite_customers
    .merge(calls, on='customer_id', how='inner')
    .merge(reason, on='call_id', how='inner')
    .merge(sentiment, on='call_id', how='inner')
)

print(elite_customers_data)

In [ ]:
elite_customers_data.head()

In [ ]:
elite_customers_data['call_start_datetime'] = pd.to_datetime(elite_customers_data['call_start_datetime'])
elite_customers_data['agent_assigned_datetime'] = pd.to_datetime(elite_customers_data['agent_assigned_datetime'])
elite_customers_data['call_end_datetime'] = pd.to_datetime(elite_customers_data['call_end_datetime'])

# Calculate HT (Handling Time) and ST (Speed of Answer Time)
elite_customers_data['HT'] = (elite_customers_data['call_end_datetime'] - elite_customers_data['call_start_datetime']).dt.total_seconds()
elite_customers_data['ST'] = (elite_customers_data['agent_assigned_datetime'] - elite_customers_data['call_start_datetime']).dt.total_seconds()


# Elite Customer vs Reasons

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate elite customer counts per reason
elite_reason_counts = elite_customers_data['primary_call_reason'].value_counts()

# Horizontal bar plot for elite customers per reason
plt.figure(figsize=(12, 8))
sns.barplot(x=elite_reason_counts.values, y=elite_reason_counts.index, palette="viridis")
plt.title('Number of Elite Customers per Primary Call Reason', fontsize=16)
plt.xlabel('Number of Elite Customers', fontsize=12)
plt.ylabel('Primary Call Reason', fontsize=12)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the benchmark values; you can replace these with actual benchmark values
ast_benchmark = 5  # Example value
aht_benchmark = 10  # Example value

# Box plot for AST
plt.figure(figsize=(12, 6))
sns.boxplot(x=elite_customers_data['ST'])
plt.axvline(ast_benchmark, color='r', linestyle='--', label='AST Benchmark')
plt.title('Box Plot of AST for Elite Customers')
plt.xlabel('AST')
plt.legend()
plt.show()

# Box plot for AHT
plt.figure(figsize=(12, 6))
sns.boxplot(x=elite_customers_data['HT'])
plt.axvline(aht_benchmark, color='r', linestyle='--', label='AHT Benchmark')
plt.title('Box Plot of AHT for Elite Customers')
plt.xlabel('AHT')
plt.legend()
plt.show()


# Agent Tone for Elite Customers

In [ ]:

plt.figure(figsize=(15, 10))

# Average Call Duration (HT) by Agent Tone
plt.subplot(2, 2, 1)
sns.barplot(data=elite_customers_data, x='agent_tone', y='HT', estimator='mean', palette='viridis')
plt.title('Average Handle Time by Agent Tone for Elite Customers')
plt.xlabel('Agent Tone')
plt.ylabel('Average Handle Time (minutes)')

